In [ ]:
!pip install skorch

In [ ]:
import os
import glob
import tqdm
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import skorch
from PIL import Image
from torchvision import transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score, accuracy_score, roc_auc_score

# Convolutional Neural Networks

## 2-D Convolution

Important parameters:
- `in_channels`: number of entering "features" e.g. color channels, data dimension
- `out_channels`: number of outgoing features
- `kernel_size`: receptive field of convolutional operator
- `stride`: step size for of convolutional operator
- `padding`: image padding to be applied before convolutional operator

How it works:

<img src='https://upload.wikimedia.org/wikipedia/commons/1/19/2D_Convolution_Animation.gif'>

ref: <a href='https://upload.wikimedia.org/wikipedia/commons/1/19/2D_Convolution_Animation.gif'>Wikimedia Commons</a>

## Example: Cat Laplacian

In [ ]:
os.system("wget https://upload.wikimedia.org/wikipedia/commons/3/3c/IC_Blue_Melody_Flipper_CHA_male_EX1_CACIB.jpg")

def image_loader(path):

  imsize = 256
  # Image loading
  # 1. Resize image so that smaller dim = imsize
  # 2. Center crop image along larger dimension to make imsize x imsize
  # 3. Convert to Pytorch tensor
  loader = transforms.Compose([transforms.Resize(imsize), 
                               transforms.CenterCrop(imsize),
                               transforms.ToTensor()])
  img = Image.open(path)
  img = loader(img).requires_grad_(True).unsqueeze(0)
  return img

In [ ]:
img = image_loader('IC_Blue_Melody_Flipper_CHA_male_EX1_CACIB.jpg')
img_py = transforms.ToPILImage()(img[0])
display(img_py)

In [ ]:
# initialize a Conv2d object for the Laplacian operator
lapl = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, stride=1, bias=False)

# set the weights of the Conv2d to match the Laplacian operator
# see: https://en.wikipedia.org/wiki/Discrete_Laplace_operator
lapl.weight.data = torch.tensor([[0, 1, 0], [1, -4, 1], [0, 1, 0]]).reshape(3, 3, 1, 1).float()

# apply the operator and view the image
cat_lapl = lapl(img)
cat_lapl_py = transforms.ToPILImage()(cat_lapl[0])
display(cat_lapl_py)

## Building a Convolutional Neural Network

<img src='https://pythonmachinelearning.pro/wp-content/uploads/2017/09/lenet-5.png.webp'>

Ref: LeCun Y, Bottou L, Bengio Y, Haffner P. <a href='https://ieeexplore.ieee.org/abstract/document/726791'>Gradient-based learning applied to document recognition</a>. Proceedings of the IEEE., 86(11), 2278-2324 (1998).



## Example: Classifying Phases of Matter

### Load dataset

In [ ]:
def load_dataset():

  # download files
  if not os.path.exists("Ising2DFM_reSample_L40_T=All.pkl") or not os.path.exists("Ising2DFM_reSample_L40_T=All_labels.pkl"):
    os.system("wget https://physics.bu.edu/~pankajm/ML-Review-Datasets/isingMC/Ising2DFM_reSample_L40_T=All.pkl")
    os.system("wget https://physics.bu.edu/~pankajm/ML-Review-Datasets/isingMC/Ising2DFM_reSample_L40_T=All_labels.pkl")

  #X = pickle.load(open('Ising2DFM_reSample_L40_T=All.pkl', 'rb'))
  # read data: 40x40 images, -1 = spin down, +1 = spin up
  X = pd.read_pickle('Ising2DFM_reSample_L40_T=All.pkl')
  X = np.unpackbits(X).reshape(-1, 1, 40, 40)
  X = X.astype('int')
  X[np.where(X==0)] = -1

  #y = pickle.load(open('Ising2DFM_reSample_L40_T=All_labels.pkl', 'rb'))
  # read labels: 0 = disordered, 1 = ordered
  y = pd.read_pickle('Ising2DFM_reSample_L40_T=All_labels.pkl')

  # Create ordered/disordered dataset
  X_ordered=X[:70000,:]
  y_ordered=y[:70000]
  X_critical=X[70000:100000,:]
  y_critical=y[70000:100000]
  X_disordered=X[100000:,:]
  y_disordered=y[100000:]
  X = np.concatenate((X_ordered, X_disordered))
  y = np.concatenate((y_ordered, y_disordered))

  # pick random data points from ordered and disordered states to create the training and test sets
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

  return torch.tensor(X_train).float(), torch.tensor(X_test).float(), \
          torch.tensor(y_train), torch.tensor(y_test)

In [ ]:
X_train, X_test, y_train, y_test = load_dataset()

### Plot some representative samples

In [ ]:
f, axes = plt.subplots(1, 4)
f.set_size_inches((16, 4))
axes[0].imshow(X_train.squeeze().numpy()[0], vmin=-1, vmax=1)
axes[1].imshow(X_train.squeeze().numpy()[1], vmin=-1, vmax=1)
axes[2].imshow(X_train.squeeze().numpy()[2], vmin=-1, vmax=1)
axes[3].imshow(X_train.squeeze().numpy()[3], vmin=-1, vmax=1)
plt.tight_layout()
plt.show()
plt.close()

### What happens when we apply `Conv2d()`? 

In [ ]:
sample = X_train[1:2]
conv = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=5, stride=2)
sample = F.pad(sample, (2, 2, 2, 2), 'circular').float()
out = conv(sample)

f, axes = plt.subplots(1, 2)
f.set_size_inches((8, 4))
axes[0].imshow(sample.squeeze().numpy(), vmin=-1, vmax=1)
axes[1].imshow(out.detach().squeeze().numpy()[40], vmin=-1, vmax=1)
plt.tight_layout()
plt.show()
plt.close()

### Build the Convolutional Neural Network

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.DIM = 16
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=self.DIM, \
                               kernel_size=5, stride=2)
        self.bn1 = nn.BatchNorm2d(self.DIM)
        self.conv2 = nn.Conv2d(in_channels=self.DIM, out_channels=2*self.DIM, \
                               kernel_size=5, stride=2)
        self.bn2 = nn.BatchNorm2d(2*self.DIM)
        self.conv3 = nn.Conv2d(in_channels=2*self.DIM, out_channels=4*self.DIM, \
                               kernel_size=5, stride=2)
        self.bn3 = nn.BatchNorm2d(4*self.DIM)
        self.pool = nn.AvgPool2d(kernel_size=5)
        self.fc1 = nn.Linear(4*self.DIM, 2*self.DIM)
        self.fc2 = nn.Linear(2*self.DIM, self.DIM)
        self.fc3 = nn.Linear(self.DIM, 2)
    
    def pad(self, x):
        # circular padding = periodic boundary condition
        return F.pad(x, (2,2,2,2), 'circular')

    def forward(self, x):
        # input: BATCH_SIZE x 1 x 40 x 40
        # layer 1: BATCH_SIZE x DIM x 20 x 20        
        out = F.relu(self.bn1(self.conv1(self.pad(x))))
        # layer 2: BATCH_SIZE x 2*DIM x 10 x 10
        out = F.relu(self.bn2(self.conv2(self.pad(out))))
        # layer 3: BATCH_SIZE x 4*DIM x 5 x 5
        out = F.relu(self.bn3(self.conv3(self.pad(out))))
        # layer 4: BATCH_SIZE x 4*DIM
        out = self.pool(out).reshape(-1, 4*self.DIM)
        # layer 5: BATCH_SIZE x 2*DIM
        out = F.dropout(F.relu(self.fc1(out)), p=0.2, training=self.training)
        # layer 6: BATCH_SIZE x DIM
        out = F.dropout(F.relu(self.fc2(out)), p=0.2, training=self.training)
        # layer 7: BATCH_SIZE x 2
        out = self.fc3(out)
        return F.softmax(out, dim=1)

    def return_activations(self, x):
        activations = []
        # input: BATCH_SIZE x 1 x 40 x 40
        # layer 1: BATCH_SIZE x DIM x 20 x 20        
        out = F.relu(self.bn1(self.conv1(self.pad(x))))
        activations.append(out)
        # layer 2: BATCH_SIZE x 2*DIM x 10 x 10
        out = F.relu(self.bn2(self.conv2(self.pad(out))))
        activations.append(out)
        # layer 3: BATCH_SIZE x 4*DIM x 5 x 5
        out = F.relu(self.bn3(self.conv3(self.pad(out))))
        activations.append(out)
        return activations

In [ ]:
from skorch import NeuralNetClassifier
model = Net()
clf = NeuralNetClassifier(model, batch_size=64, max_epochs=2, lr=1e-3, device='cuda')
clf.fit(X_train, y_train)

In [ ]:
clf.score(X_test, y_test)

### Visualizing the layer-by-layer activations

In [ ]:
i = 1
example = X_train[i:i+1].to('cuda')
print(example.shape)
activations = clf.module_.return_activations(example)
f, axes = plt.subplots(1, 4)
f.set_size_inches((16, 4))
axes[0].imshow(example.cpu().squeeze().numpy(), vmin=-1, vmax=1)
axes[1].imshow(activations[0][0, np.random.randint(16), :, :].detach().cpu().squeeze().numpy(), vmin=-1, vmax=1)
axes[2].imshow(activations[1][0, np.random.randint(32), :, :].detach().cpu().squeeze().numpy(), vmin=-1, vmax=1)
axes[3].imshow(activations[2][0, np.random.randint(64), :, :].detach().cpu().squeeze().numpy(), vmin=-1, vmax=1)
plt.tight_layout()
plt.show()
plt.close()